In [ ]:
n_epchs= 3
counter=1
lst_loss=[]
lst_acc=[]
lst_reports=[]
lst_AUC=[]
lst_matrix=[]
lst_histories=[]

In [ ]:
from sklearn.model_selection import KFold
kfold=KFold(5,random_state=0,shuffle=True)
for train,test in kfold.split(x_data,y_data):
 X_train = x_data[train]
 X_test = x_data[test]
 y_train = y_data[train]
 y_test = y_data[test]



X_train = np.reshape(X_train, (len(X_train), 100, 100))
X_test = np.reshape(X_test, (len(X_test), 100, 100))

In [ ]:
from keras.utils import np_utils
y_train = np_utils.to_categorical(y_train, 5)
y_test = np_utils.to_categorical(y_test, 5)

In [ ]:

from keras.models import Sequential
from keras.layers import Input,Dense,Flatten,Conv1D,MaxPool1D,Dropout,Activation,BatchNormalization
from keras.optimizers import Adam
from keras.losses import categorical_crossentropy
from keras.regularizers import l1,l2

model=Sequential()
model.add((Conv1D(filters=32,kernel_size=3,padding='same',strides=2,activation='relu',input_shape=(100,100))))
#model.add(MaxPool1D(2))
model.add((Conv1D(filters=64,kernel_size=3,strides=1,activation='relu')))
#model.add(MaxPool1D(2))
model.add((Conv1D(filters=128,kernel_size=3,strides=1,activation='relu')))
#model.add(MaxPool1D(2))
model.add((Conv1D(filters=256,kernel_size=3,strides=1,activation='relu')))
model.add((Conv1D(filters=256,kernel_size=3,strides=1,activation='relu')))
#model.add(MaxPool1D(2))
model.add(Flatten())
# model.add(Dense(64,activation='relu'))
model.add(Dense(32,activation='relu'))
model.add(Dense(16,activation='relu'))
model.add(Dense(5,activation='softmax'))
model.compile(loss=categorical_crossentropy,optimizer=Adam(),metrics=['accuracy'])

'''
from keras.utils import plot_model
plot_model(model,to_file='saman1.jpeg',show_shapes= True)
'''
print(model.summary())

In [ ]:
history = model.fit(X_train, y_train,validation_data=(X_test,y_test),batch_size=512,epochs=n_epchs)
lst_histories.append(history)

In [ ]:
  test_loss, test_acc=model.evaluate(X_test,y_test)
  lst_loss.append(test_loss)
  lst_acc.append(test_acc)

  # 1 = loss: 0.2850 - accuracy: 0.9329
  # 2 = loss: 0.0792 - accuracy: 0.9765

In [ ]:
  from sklearn.metrics import roc_curve, auc, roc_auc_score
  predicts=model.predict(X_test)
  predicts=predicts.argmax(axis=1)
  actuals=y_test.argmax(axis=1)

#   fpr,tpr,thr=roc_curve(actuals,predicts)
#   a=auc(fpr,tpr)
#   lst_AUC.append(a)

  r=classification_report(actuals,predicts)
  lst_reports.append(r)

  c=confusion_matrix(actuals,predicts)
  lst_matrix.append(c)

  counter+=1


In [ ]:
NetPlot(lst_histories,n_epchs)


acc_avg=str(np.sum(lst_acc)/5)
loss_avg=str(np.sum(lst_loss)/5)
AUC_avg=str(np.sum(lst_AUC)/5)

acc=str(lst_acc)
loss=str(lst_loss)
AUC=str(lst_AUC)

results_path='/content/results_CNN_pd.txt'
f1=open(results_path,'a')
f1.write('Avergae Accuracy: '+acc_avg+'\n'+'Average Loss: '+loss_avg+'\n'+'Average AUc: '+AUC_avg+'\n')
f1.write('\nAccuracies: '+acc+'\n'+'Losses: '+loss+'\n'+'AUCs: '+AUC+'\n')
f1.write('\n\nMetrics for all folds: \n\n')
for i in range(len(lst_reports)):
    f1.write(str(lst_reports[i]))
    f1.write('\n\nCofusion Matrix: \n'+str(lst_matrix[i])+'\n\n__________________________________________________________\n')

f1.close()

In [ ]:
import numpy as np
from scipy import interp
import matplotlib.pyplot as plt
from itertools import cycle
from sklearn.metrics import roc_curve, auc

y_score = model.predict(X_test)
# Plot linewidth.
lw = 1
n_classes = 5
# Compute ROC curve and ROC area for each class
fpr = dict()
tpr = dict()
roc_auc = dict()
for i in range(n_classes):
    fpr[i], tpr[i], _ = roc_curve(y_test[:, i], y_score[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])

# Compute micro-average ROC curve and ROC area
fpr["micro"], tpr["micro"], _ = roc_curve(y_test.ravel(), y_score.ravel())
roc_auc["micro"] = auc(fpr["micro"], tpr["micro"])

# Compute macro-average ROC curve and ROC area

# First aggregate all false positive rates
all_fpr = np.unique(np.concatenate([fpr[i] for i in range(n_classes)]))

# Then interpolate all ROC curves at this points
mean_tpr = np.zeros_like(all_fpr)
for i in range(n_classes):
    mean_tpr += interp(all_fpr, fpr[i], tpr[i])

# Finally average it and compute AUC
mean_tpr /= n_classes

fpr["macro"] = all_fpr
tpr["macro"] = mean_tpr
roc_auc["macro"] = auc(fpr["macro"], tpr["macro"])

# Plot all ROC curves
plt.figure(1)
#plt.plot(fpr["micro"], tpr["micro"],
        # label='micro-average ROC curve (area = {0:0.2f})'
 #              ''.format(roc_auc["micro"]),
  #       color='deeppink', linestyle=':', linewidth=4)

#plt.plot(fpr["macro"], tpr["macro"],
        # label='macro-average ROC curve (area = {0:0.2f})'
 #              ''.format(roc_auc["macro"]),
  #       color='navy', linestyle=':', linewidth=4)

colors = cycle(['aqua', 'darkorange', 'cornflowerblue'])
for i, color in zip(range(n_classes), colors):
    plt.plot(fpr[i], tpr[i], color=color, lw=lw,
             label='ROC curve of class {0} (area = {1:0.2f})'
             ''.format(i, roc_auc[i]))

plt.plot([0, 1], [0, 1], 'k--', lw=lw)
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
#plt.title('Some extension of Receiver operating characteristic to multi-class')
plt.legend(loc="lower right")
plt.savefig('roc')
plt.show()


# Zoom in view of the upper left corner.
plt.figure(2)
plt.xlim(0, 0.2)
plt.ylim(0.8, 1)
#plt.plot(fpr["micro"], tpr["micro"],
         #label='average ROC curve (area = {0:0.2f})'
 #              ''.format(roc_auc["micro"]),
  #       color='deeppink', linestyle=':', linewidth=4)

#plt.plot(fpr["macro"], tpr["macro"],
         #label='macro-average ROC curve (area = {0:0.2f})'
 #              ''.format(roc_auc["macro"]),
  #       color='navy', linestyle=':', linewidth=4)

colors = cycle(['aqua', 'darkorange', 'cornflowerblue'])
for i, color in zip(range(n_classes), colors):
    plt.plot(fpr[i], tpr[i], color=color, lw=lw,
             label='ROC curve of class {0} (area = {1:0.2f})'
             ''.format(i, roc_auc[i]))

plt.plot([0, 1], [0, 1], 'k--', lw=lw)
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
#plt.title('Some extension of Receiver operating characteristic to multi-class')
plt.legend(loc="lower right")
plt.savefig('roc_zoom')
plt.show()
